# Llama3 Quantization

[Llama3 Quantization](https://www.reddit.com/r/LocalLLaMA/comments/1cci5w6/quantizing_llama_3_8b_seems_more_harmful_compared/)
[How Good Are Low-bit Quantized LLaMA3 Models?]("https://arxiv.org/abs/2404.14047v1")

[llama-cpp-python]("https://github.com/abetlen/llama-cpp-python")

In [1]:
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu121
     ---------------------------------------- 0.0/201.3 MB ? eta -:--:--
     -------------------------------------- 0.0/201.3 MB 660.6 kB/s eta 0:05:05
     ---------------------------------------- 0.3/201.3 MB 3.5 MB/s eta 0:00:57
     --------------------------------------- 1.5/201.3 MB 11.6 MB/s eta 0:00:18
     --------------------------------------- 2.6/201.3 MB 14.9 MB/s eta 0:00:14
      -------------------------------------- 4.0/201.3 MB 18.4 MB/s eta 0:00:11
     - ------------------------------------- 5.7/201.3 MB 21.2 MB/s eta 0:00:10
     - ------------------------------------- 7.3/201.3 MB 23.5 MB/s eta 0:00:09
     - ------------------------------------- 8.9/201.3 MB 24.8 MB/s eta 0:00:08
     - ------------------------------------ 10.2/201.3 MB 25.0 MB/s eta 0:00:08
     -- ----------------------------------- 11.8/201.3 MB 31.2 MB/s eta 0:00:07
     -- ---------------------

In [1]:
from llama_cpp import Llama

In [26]:
model_id = "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF"

llm = Llama.from_pretrained(
    repo_id=model_id,
    filename="*Q4_K_M.gguf",
    #chat_format="llama-3",
    verbose=False
)

## Chat Recommendation Example

In [27]:
{
    "topic": "Family Bonding Activities",
    "interests": ["Sports", "Family", "Bonding"],
    "next_question": "저는 최근에 가족들과 함께하는 새로운 활동을 찾고 있습니다. 혹시 B님은 가족들과 함께하는 다른 활동을 추천해주실 수 있나요?",
    "follow_up": "예를 들어, 등산이나 캠핑 등 야외 활동을 함께 하시나요?"
}

{'topic': 'Family Bonding Activities',
 'interests': ['Sports', 'Family', 'Bonding'],
 'next_question': '저는 최근에 가족들과 함께하는 새로운 활동을 찾고 있습니다. 혹시 B님은 가족들과 함께하는 다른 활동을 추천해주실 수 있나요?',
 'follow_up': '예를 들어, 등산이나 캠핑 등 야외 활동을 함께 하시나요?'}

## Chat History Preset

In [28]:
from json import load

In [29]:
class ChatPresets(list):
    preset_list = [
        "chat_presets/conversation1.json",
        "chat_presets/conversation2.json",
        "chat_presets/conversation3.json",
        "chat_presets/conversation4.json"
    ]
    
    def __init__(self):
        super().__init__()
        for preset in ChatPresets.preset_list:
            with open(preset, "r", encoding="UTF-8") as f:
                self.extend(load(f)['conversations'])
    
    @staticmethod
    def create_prompt(system_prompt: str, messages: list, user_prompt: str = ""):
        return [
            {
                "role": "system",
                "content": system_prompt
            },
            *messages,
            {
                "role": "user",
                "content": user_prompt
            }
        ]
    
    def __getitem__(self, key):
        if isinstance(key, int):
            return super().__getitem__(key)['messages']
        else:
            return self.create_prompt(key[0], self[key[1]], key[2] if len(key) > 2 else "")

ChatPresets = ChatPresets()

In [30]:
# Test
for line in ChatPresets[0]:
    print(line)

ChatPresets["시스템 프롬포트", 4]

{'role': 'A', 'content': '요즘 가족들이 잘 지내고 계신가요?'}
{'role': 'B', 'content': '네, 가족들은 다 잘 지내고 있어요.'}
{'role': 'A', 'content': '저는 아이들과 스포츠를 즐기는걸 좋아해요. 어떤 종류의 스포츠를 함께 하시나요?'}
{'role': 'B', 'content': '주로 축구와 테니스를 함께 하고 있어요. 아이들이 적극적으로 참여하면서 즐거운 시간을 보내고 있어요.'}
{'role': 'A', 'content': '축구와 테니스는 정말 가족끼리 함께하기 좋은 스포츠죠. 활동적인 시간을 보내면서 가족 간의 유대감도 높일 수 있을 거예요. 다음에는 함께 스포츠를 즐기며 가족끼리 더 많은 시간을 보내는 건 어떨까요?'}
{'role': 'B', 'content': '네, 좋아요! 함께 활동을 하면서 가족 간의 유대감을 높이는 건 정말 좋은 생각이에요.'}


[{'role': 'system', 'content': '시스템 프롬포트'},
 {'role': 'A', 'content': '요즘 가족들이 잘 지내고 계신가요?'},
 {'role': 'B', 'content': '네, 가족들은 다 잘 지내고 있어요. 그런데 최근에는 취업 문제로 걱정이 좀 되네요.'},
 {'role': 'A', 'content': '취업 문제로 걱정이 되시나봐요. 어떤 일이 있으신 거죠?'},
 {'role': 'B', 'content': '제가 원하는 분야에서 일자리를 찾는 게 쉽지 않아서 좀 고민이 되요.'},
 {'role': 'A', 'content': '취업은 정말 어려운 문제죠. 어떤 분야에 지원하시나요?'},
 {'role': 'B', 'content': '저는 IT 분야에 관심이 많아서 그쪽에서 일을 찾고 있어요. 현재는 노력 중이에요.'},
 {'role': 'user', 'content': ''}]

## Recommendation for the next conversation based on the current conversation

##### 한번에 출력

In [31]:
system_type_1 = "당신은 한국어로 답변하는 소개팅 전문가 입니다. A와 B 두 사람의 대화를 계속해서 듣고, A의 입장에서 대화를 이어나갈 주제를 자세하게 추천해주어야 합니다. 대화 내용의 추천은 json 형태를 따라야 합니다."

In [32]:
response = llm.create_chat_completion(
    messages=ChatPresets[system_type_1, 4],
    response_format={"type": "json_object"}
)

In [33]:
print(response["choices"][0]["message"]["content"])

{
"topic": "career",
"subtopics": [
  {
    "question": "A: 취업 문제로 걱정이 되시나봐요. 어떤 일이 있으신 거죠?",
    "answer": "B: 저는 원하는 분야에서 일자리를 찾는 게 쉽지 않아서 좀 고민이 되요."
  },
  {
    "question": "A: 취업은 정말 어려운 문제죠. 어떤 분야에 지원하시나요?",
    "answer": "B: 저는 IT 분야에 관심이 많아서 그쪽에서 일을 찾고 있어요. 현재는 노력 중이에요."
  }
]
}


##### 스트림 출력

In [34]:
system_type_2 = "You are a helpful, smart, kind, and efficient Conversation Analysis and Recommendation AI System. You always fulfill the user's requests to the best of your ability. You need to keep listen to the conversations. Please answer in Korean language."
user_prompt_2 = "Based on the conversations between A and B, on be half of A, do recommend a new topic sentence related the current situation."

In [35]:
response = llm.create_chat_completion(
    messages=ChatPresets[system_type_2, 4, user_prompt_2],
    temperature=0.6,
    stream=True
)

In [36]:
for chunk in response:
    delta = chunk["choices"][0]["delta"]
    if "content" not in delta:
        continue
    print(delta["content"], end="", flush=True)
print()

I'd like to recommend a new topic sentence that's related to the current situation:

"IT 분야에 지원하고 있는 중이라면, 어떤 특정 기술이나 스킬을 배워보는 것이 도움이 될까요?"

(This translates to: "Since you're job-hunting in the IT field, do you think learning a specific technology or skill would be helpful?")


##### 반복 테스트

In [37]:
from tqdm.notebook import tqdm

In [39]:
for i in tqdm(range(5, 10)):
    message = ChatPresets[system_type_2, i, user_prompt_2]
    print("PROMPT:")
    for line in message:
        print(line)
    print()
    
    response = llm.create_chat_completion(
        messages=message,
        #temperature=0.6,
        stream=True
    )

    for chunk in response:
        delta = chunk["choices"][0]["delta"]
        if "content" not in delta:
            continue
        print(delta["content"], end="", flush=True)
    print("\n\n")

  0%|          | 0/5 [00:00<?, ?it/s]

PROMPT:
{'role': 'A', 'content': '요즘 가족들이 잘 지내고 계신가요?'}
{'role': 'B', 'content': '네, 가족들은 다 잘 지내고 있어요.'}
{'role': 'A', 'content': '저는 아이들과 스포츠를 즐기는걸 좋아해요. 어떤 종류의 스포츠를 함께 하시나요?'}
{'role': 'B', 'content': '주로 축구와 테니스를 함께 하고 있어요. 아이들이 적극적으로 참여하면서 즐거운 시간을 보내고 있어요.'}
{'role': 'A', 'content': '축구와 테니스는 정말 가족끼리 함께하기 좋은 스포츠죠. 활동적인 시간을 보내면서 가족 간의 유대감도 높일 수 있을 거예요. 다음에는 함께 스포츠를 즐기며 가족끼리 더 많은 시간을 보내는 건 어떨까요?'}
{'role': 'B', 'content': '네, 좋아요! 함께 활동을 하면서 가족 간의 유대감을 높이는 건 정말 좋은 생각이에요.'}

Here's a recommended topic sentence:

"다음 주에 경기를 볼 때는 어디서 가서 보려면 좋을까요?"

(This translates to "Do you think we should go watch the game at someone else's place next week?")

This sentence is relevant to the current conversation about watching sports and enjoying food while doing so. It also opens up a new topic for discussion, allowing the conversation to flow naturally into a planning phase for their future plans.


PROMPT:
{'role': 'A', 'content': '요즘 가족들이 잘 지내고 계신가요?'}
{'role': 'B', 'content': '네, 가족들은 다 잘 지내고 있

## Conversation Analysis

In [15]:
system_type_3 = "You are a helpful, smart, kind, and efficient Conversation Analysis and Recommendation Artificial Intelligence that outputs in JSON. You always fulfill the user's requests to the best of your ability. You need to keep listen the conversations between A and B, and analyze the conversation."

In [16]:
schema = {
    "type": "object",
    "properties": {
        "topic": {"type": "string"},
        "interests": {"type": "list", "items": {"type": "string"}},
        "initiative": {"type": "list", "items": {"type": "percentage"}},
        "favorable": {"type": "list", "items": {"type": "percentage"}}
    },
    "required": ["topic", "interests"],
}

In [17]:
response = llm.create_chat_completion(
    messages=ChatPresets[system_type_3, 4],
    response_format={
        "type": "json_object",
        "schema": schema,
    },
    temperature=0.2,
)

In [18]:
print(response["choices"][0]["message"]["content"])

{ "status": "ready", "message": "I'm ready to help! Please go ahead with the conversation." }


##### 반복 테스트

In [19]:
from tqdm.notebook import tqdm

In [20]:
for i in tqdm(range(5, 10)):
    message = ChatPresets[system_type_2, i]
    print("PROMPT:")
    for line in message:
        print(line)
    print()

    response = llm.create_chat_completion(
        messages=ChatPresets[system_type_3, i],
        response_format={
            "type": "json_object",
            "schema": schema,
        },
        temperature=0.2,
    )

    print(response["choices"][0]["message"]["content"], "\n\n", flush=True)

  0%|          | 0/5 [00:00<?, ?it/s]

PROMPT:
{'role': 'A', 'content': '요즘 가족들이 잘 지내고 계신가요?'}
{'role': 'B', 'content': '네, 가족들은 다 잘 지내고 있어요.'}
{'role': 'A', 'content': '저는 아이들과 스포츠를 즐기는걸 좋아해요. 어떤 종류의 스포츠를 함께 하시나요?'}
{'role': 'B', 'content': '주로 축구와 테니스를 함께 하고 있어요. 아이들이 적극적으로 참여하면서 즐거운 시간을 보내고 있어요.'}
{'role': 'A', 'content': '축구와 테니스는 정말 가족끼리 함께하기 좋은 스포츠죠. 활동적인 시간을 보내면서 가족 간의 유대감도 높일 수 있을 거예요. 다음에는 함께 스포츠를 즐기며 가족끼리 더 많은 시간을 보내는 건 어떨까요?'}
{'role': 'B', 'content': '네, 좋아요! 함께 활동을 하면서 가족 간의 유대감을 높이는 건 정말 좋은 생각이에요.'}


{ "status": "ready", "message": "I'm ready to help! Please go ahead with the conversation." } 

PROMPT:
{'role': 'A', 'content': '요즘 가족들이 잘 지내고 계신가요?'}
{'role': 'B', 'content': '네, 가족들은 다 잘 지내고 있어요.'}
{'role': 'A', 'content': '저는 아이들과 스포츠를 즐기는걸 좋아해요. 어떤 종류의 스포츠를 함께 하시나요?'}
{'role': 'B', 'content': '주로 축구와 테니스를 함께 하고 있어요. 아이들이 적극적으로 참여하면서 즐거운 시간을 보내고 있어요.'}
{'role': 'A', 'content': '축구와 테니스는 정말 가족끼리 함께하기 좋은 스포츠죠. 활동적인 시간을 보내면서 가족 간의 유대감도 높일 수 있을 거예요. 다음에는 함께 스포츠를 즐기며 가족끼리 더 많은 시간을 보내는 건 어떨까요?'}
{'role': 'B', 'cont